<a href="https://colab.research.google.com/github/saigogula/Suicide-Detection/blob/main/SuicideDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import io
import os
import torch
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score


In [5]:
from google.colab import files
uploaded = files.upload()

Saving archive.zip to archive.zip


In [ ]:
zip_file = uploaded['archive.zip']
with zipfile.ZipFile(io.BytesIO(zip_file), 'r') as zip_ref:
    zip_ref.extractall('dataset')

In [ ]:
data_path = os.path.join('dataset', 'Suicide_Detection.csv')
df = pd.read_csv(data_path)

In [ ]:
df = df[['text', 'class']]
df['class'] = df['class'].map({'suicide': 1, 'non-suicide': 0})

<ipython-input-13-9f1f602a3490>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['class'] = df['class'].map({'suicide': 1, 'non-suicide': 0})


In [ ]:
train_texts, val_texts, train_classes, val_classes = train_test_split(df['text'].tolist(), df['class'].tolist(), test_size=0.2)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)

In [ ]:
class SuicideDetectionDataset(Dataset):
    def __init__(self, encodings, classes):
        self.encodings = encodings
        self.classes = classes

    def __len__(self):
        return len(self.classes)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.classes[idx])
        return item

In [ ]:
train_dataset = SuicideDetectionDataset(train_encodings, train_classes)
val_dataset = SuicideDetectionDataset(val_encodings, val_classes)

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50
)

training_args = TrainingArguments(
output_dir="./checkpoints",
save_strategy="steps",
save_steps=1000,
save_total_limit=3,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,    )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import os

checkpoint_dir = './checkpoints/content/checkpoints/runs/'
os.makedirs(checkpoint_dir, exist_ok=True)

In [ ]:
import os

checkpoint_dir = './checkpoints/content/checkpoints/runs/'
print(os.listdir(checkpoint_dir))

[]


In [ ]:
!ls ./checkpoints/runs/Sep22_17-23-53_980318948d5e

events.out.tfevents.1727026070.980318948d5e.4240.0


In [ ]:
trainer.train()

Step,Training Loss


In [ ]:
training_logs = trainer.state.log_history

In [ ]:
eval_steps = []
accuracy_vals = []

for log in training_logs:
    if 'eval_accuracy' in log:
        eval_steps.append(log['step'])
        accuracy_vals.append(log['eval_accuracy'])

In [ ]:
plt.plot(eval_steps, accuracy_vals, label='Accuracy')
plt.xlabel('Evaluation Step')
plt.ylabel('Accuracy')
plt.title('Model Accuracy over Time')
plt.legend()
plt.show()

In [ ]:
model.save_pretrained('./suicide_detection_model')

In [ ]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1)
    return "Suicidal" if prediction.item() == 1 else "Non-Suicidal"

In [ ]:
test_text = "I don't want to live anymore"
print(predict(test_text))